In [4]:
#Import the necessary functions
import cv2
import numpy as np
import os
import argparse

In [ ]:
#Function which is used to generate mask of the input image
#Input: src_file- path to the input image
#       dst_file- path to where the mask will be stored
# Hyperparameters:
#     gau_block_size - Block Size for Adaptive Gaussian Threshold
#     gau_cons - Constant passed to Adaptive Gaussian Threshold
#     otsu_ratio - The constant which takes value between 0 to 1, and is multiplied with the threshold
#                  returned by otsu threshold method
# gau_blur - The number of times gaussian kernel of sigma = 1 to be applied.

def get_mask(src_file, dst_file, gau_block_size=427, gau_cons=13, otsu_ratio=0.6, gau_blur=100):
    
    #Load the input image
    src_img = cv2.imread(src_file, 0)
    img = src_img.copy()
    
    #Apply gaussian blur, 100 iterations mean sigma = 10
    for i in range(gau_blur):
        img = cv2.GaussianBlur(img, (3, 3), 0)
    
    #Adaptive Gaussian Threshold
    gaussian_binary = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY, gau_block_size, gau_cons)
    
    #Otsu's Threshold
    otsu_high_thresh, _ = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    _, otsu_binary = cv2.threshold(img, otsu_ratio*otsu_high_thresh, 255, cv2.THRESH_BINARY)

    # Logical and between the output of above two thresholds
    binary = (np.logical_and(otsu_binary, gaussian_binary)*255).astype('uint8')

    #Laplacian on the binary image
    lap_out = cv2.Laplacian(binary, -1, 3)
    
    #Find all the contours generated by the laplacian
    contours, hierarchy = cv2.findContours(lap_out.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    #Go through all the contours, calculate their area and find the contour with maximum area in a range.
    #The range is determined by the size of the input image.
    m = -1
    max_area = -1
    for i, cont in enumerate(contours):
        area = cv2.contourArea(cont)
        if((area > max_area) and (area < 0.1*src_img.size)):
            max_area = area
            m = i

    #draw the contour found[Gall bladder] on a blank canvas. Fill the entire contour
    mask = np.zeros(src_img.shape+(3, ), dtype='uint8')
    cv2.drawContours(mask, [contours[m]], -1, (255,255,255), -1)
    
    #Saving the mask at the desired destination 
    cv2.imwrite(dst_file, mask)

In [ ]:
if __name__ == "__main__":
    
    #Describing command line usage
    parser = argparse.ArgumentParser(description='Script for generating segmentation mask of gall bladder images')
    parser.add_argument('-i', '--img_path', type=str, default='img', required=True, help="Path to the image folder")
    parser.add_argument('-d', '--det_path', type=str, default='val_masks', required=True, help="Path to the folder where mask will be stored")
    
    #Parsing folder in which image is contained and the destination folder where the image has to be stored
    args = parser.parse_args()
    src_folder =  args.img_path
    dst_folder  = args.det_path
    
    #All the images in the source directory
    src_files = os.listdir(src_folder)
    
    #If the destination directory doesn't exist then create it
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
        
    #Generate mask for all the imput images
    for src_file in src_files:
        file = src_file[:-4] + ".png"
        get_mask(src_folder+src_file, dst_folder+file)
        
    print("Done!")

In [ ]:
# ######  All the code down below belongs to the experiments done, and hence is commented out

# #Using python's multiprocessing library for grid search in a parllel computing manner.
# from multiprocessing import Process
# import multiprocessing

# process = []
# manager = multiprocessing.Manager()
# accuracies = manager.dict()
# parallel_processes = 5

# count = 0
# for gau_block_size in range(101, 501, 10):
#     for gau_cons in range(5, 15, 1):
#         for otsu_ratio in range(30, 90, 5):
#             otsu_ratio = otsu_ratio/100
#             #the arguments of the get_mask functions are changed, this line will have to be edited and the
#             #function also will have to make necessary changes
#             p = Process(target=get_mask, args=(gau_block_size, gau_cons, otsu_ratio, accuracies))
#             p.start()
#             process.append(p)

#             count += 1
#             if(count%parallel_processes == 0):
#                 for p in process:
#                     p.join()
#                 process = []

# print(max(accuracies, key= lambda x: accuracies[x]))

In [5]:
# # All the image filtering techniques that were tried

# def image_filtering(src_file, dst_file):
    
#     src_img = cv2.imread(src_file, 0)
    
#     img = src_img.copy()     ##Gaussian Blur
#     for i in range(100):
#         img = cv2.GaussianBlur(img, (3, 3), 0)       
#     cv2.imwrite("gau_"+dst_file, img)
    
#     img = src_img.copy()     ## Average Blur
#     for i in range(100):
#         img = cv2.blur(img, (3, 3))
#     cv2.imwrite("avg_"+dst_file, img)
    
#     img = src_img.copy()
#     for i in range(100):    ## Median Filtering
#         img = cv2.medianBlur(img, 3)    
#     cv2.imwrite("median_"+dst_file, img)
    
#     img = src_img.copy()
#     for i in range(100):    ## bilateral Filtering
#         img = cv2.bilateralFilter(img, 3, 5, 5)
#     cv2.imwrite("bilateral_"+dst_file, img)

In [8]:
# # All the image thresholding techniques that were tried

# def image_thresholding(src_file, dst_file):
    
#     src_img = cv2.imread(src_file, 0)
    
#       #Gaussian Blur
#     img = src_img.copy()
#     for i in range(100):
#         img = cv2.GaussianBlur(img, (3, 3), 0)  
        
#       #Adaptive Gaussian Threshold
#     gaussian_binary = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                             cv2.THRESH_BINARY, 437, 13)
#     cv2.imwrite("gaussian_"+dst_file, gaussian_binary)
      
#       #Adaptive Mean Threshold
#     mean_binary = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
#                                             cv2.THRESH_BINARY, 437, 13)
#     cv2.imwrite("mean_"+dst_file, mean_binary)
        
#       #Otsu Threshold
#     otsu_high_thresh, _ = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     _, otsu_binary = cv2.threshold(img, 0.6*otsu_high_thresh, 255, cv2.THRESH_BINARY)
#     cv2.imwrite("otsu_"+dst_file, otsu_binary)
    

In [ ]:
# # Canny Edge Detection method

# def canny_edge(src_file, dst_file):
#     src_img = cv2.imread(src_file, 0)
        
#     # Gaussian Blur
#     img = src_img.copy()
#     for i in range(100):
#         img = cv2.GaussianBlur(img, (3, 3), 0) 
    
#     # Otsu Threshold for thresholds of hysteresis
#     high_thresh, _ = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


#     # Canny edge method with threshold in 1:2 ratio
#     canny = cv2.Canny(img, 0.5*high_thresh, high_thresh, 5)
    
#     cv2.imwrite("canny_"+dst_file, canny)

In [47]:
# # Difference of Gaussian method

# def diff_of_gau(src_file, dst_file):
#     src_img = cv2.imread(src_file, 0)
    
#     # take Gaussian blur and calculate difference at each iteration
#     # Storing after every 10th iteration for inspection
#     img = src_img.copy()
#     for i in range(50):
#         gau = cv2.GaussianBlur(img, (3, 3), 0) 
#         if(i%10==0):
#             cv2.imwrite(str(i)+"_DoG_"+dst_file, img-gau)
#         img = gau    